**INITIAL CONFIGURATION/PROCESSING OF EXTERNAL DATABASES**\
(see 'openDataSUS\data\__external_databases\' to all information on provided data from sources other than the SRAG data set)

This notebook reads pd.DataFrame for the following data:
> IBGE (\__external_databases\ibge) data sets
    1. Population estimation of 2020 and 2021 [municipal level]
    2. PIB and PIB percapta (2018) [municipal level]
    3. General information (population, density, IDH,..) [UF level]
    4. Information selected from the national health research (2019) on cronic diseases and life style [UF level]

> AtlasBrasil (\__external_databases\atlasbrasil-org) data sets
    Information related to the Census 2010
    
The information loaded is a preliminary selection of features and no processing of the data
'''

In [1]:
import pandas as pd
import numpy as np
import unicodedata
from zipfile import ZipFile
from functools import reduce

PATH_EXTDB = "D:\\GitRepositories\\openDataSUS\\data\\__external_databases\\"

# ===================
#                IBGE
# ===================

# 1. estimativa-populacao

### estimativa_dou_2020.xls / estimativa_dou_2021.xls

In [2]:
# 2020
p = PATH_EXTDB + "ibge\\estimativa-populacao\\estimativa_dou_2020.xls"
col_names = ['UF', 'CO_UF', 'CO_MUN', 'ID_MUN', 'POP_ESTIMADA_20']
dtypes = {attr: val for attr, val in zip(col_names, [str, int, int, str, str])}
df_est = pd.read_excel(p, sheet_name=1, header=1, names=col_names, dtype=dtypes, nrows=5572-2)
# 2021
p = PATH_EXTDB + "ibge\\estimativa-populacao\\estimativa_dou_2021.xls"
col_names = ['UF', 'CO_UF', 'CO_MUN', 'ID_MUN', 'POP_ESTIMADA_21']
dtypes = {attr: val for attr, val in zip(col_names, [str, int, int, str, str])}
df21 = pd.read_excel(p, sheet_name=1, header=1, names=col_names, dtype=dtypes, nrows=5572-2)

df_est['POP_ESTIMADA_21'] = df21['POP_ESTIMADA_21']

'''OBS: (df_est['CO_MUN'] != df21['CO_MUN'])sum() >> 0'''

"OBS: (df_est['CO_MUN'] != df21['CO_MUN'])sum() >> 0"

In [3]:
df_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UF               5570 non-null   object
 1   CO_UF            5570 non-null   int32 
 2   CO_MUN           5570 non-null   int32 
 3   ID_MUN           5570 non-null   object
 4   POP_ESTIMADA_20  5570 non-null   object
 5   POP_ESTIMADA_21  5570 non-null   object
dtypes: int32(2), object(4)
memory usage: 217.7+ KB


In [4]:
df_est.head()

,UF,CO_UF,CO_MUN,ID_MUN,POP_ESTIMADA_20,POP_ESTIMADA_21
0,RO,11,15,Alta Floresta D'Oeste,22728,22516
1,RO,11,23,Ariquemes,109523,111148
2,RO,11,31,Cabixi,5188,5067
3,RO,11,49,Cacoal,85893,86416
4,RO,11,56,Cerejeiras,16204,16088


# 2. municipal_data

In [5]:
p = PATH_EXTDB + "ibge\\municipal_data\\PIB-dos-Municipios_base-de-dados-2010-2018.xls"
col_names = ['ANO', 'CO_UF', 'UF', 'ID_UF', 'CO_MUN','ID_MUN', 'PIB', 'PIB_PERCAPTA']
df = pd.read_excel(p, usecols=[0,3,4,5,6,7,38,39], names=col_names)
df = df[df.ANO==2018].reset_index(drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ANO           5570 non-null   int64  
 1   CO_UF         5570 non-null   int64  
 2   UF            5570 non-null   object 
 3   ID_UF         5570 non-null   object 
 4   CO_MUN        5570 non-null   int64  
 5   ID_MUN        5570 non-null   object 
 6   PIB           5570 non-null   float64
 7   PIB_PERCAPTA  5570 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 348.2+ KB


In [7]:
df.head()

,ANO,CO_UF,UF,ID_UF,CO_MUN,ID_MUN,PIB,PIB_PERCAPTA
0,2018,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,499305.982,21552.47
1,2018,11,RO,Rondônia,1100023,Ariquemes,2463773.260,23206.36
2,2018,11,RO,Rondônia,1100031,Cabixi,140502.269,25837.12
3,2018,11,RO,Rondônia,1100049,Cacoal,2180454.016,25708.96
4,2018,11,RO,Rondônia,1100056,Cerejeiras,472178.834,28714.35


# 3. uf general info

In [8]:
import re
p = PATH_EXTDB + "ibge\\uf_general_info.csv"
col_names = ['ID_UF', 'CO_UF', 'A_TERRITORIO_2020', 'POP_EST_2021', 'DENS_DEMO_2010', 'IDH_2010', 'REND_MENS_DOM_PERCAP_2020']
df = pd.read_csv(p, header=1, usecols=[0,1,5,6,7,9,12], names=col_names)
df['ID_UF'] = df['ID_UF'].apply(lambda v: re.sub(r'\&([a-z])\w*\;',r'\1', v))

ID_UF = df['ID_UF'].values

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID_UF                      27 non-null     object 
 1   CO_UF                      27 non-null     int64  
 2   A_TERRITORIO_2020          27 non-null     float64
 3   POP_EST_2021               27 non-null     int64  
 4   DENS_DEMO_2010             27 non-null     float64
 5   IDH_2010                   27 non-null     float64
 6   REND_MENS_DOM_PERCAP_2020  27 non-null     int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 1.6+ KB


In [10]:
df.head()

,ID_UF,CO_UF,A_TERRITORIO_2020,POP_EST_2021,DENS_DEMO_2010,IDH_2010,REND_MENS_DOM_PERCAP_2020
0,Acre,12,164173.431,906876,4.47,0.663,917
1,Alagoas,27,27830.656,3365351,112.33,0.631,796
2,Amapa,16,142470.762,877613,4.69,0.708,893
3,Amazonas,13,1559167.878,4269995,2.23,0.674,852
4,Bahia,29,564760.427,14985284,24.82,0.660,965


# 4. pesquisa-nacional-saude-2019

### Modulo N - Percepção do Estado de Saúde (por UF)
Tabelas:\
1.1. Proporção de pessoas de 18 anos ou mais de idade com autoavaliação de saúde muito
boa/boa
### Modulo P -  Estilos de Vida (por UF)
Tabelas:\
2.18. Proporção de pessoas de 18 anos ou mais de idade que praticam o nível recomendado de
atividade física no lazer\
2.26. Proporção de pessoas de 18 anos ou mais de idade usuários atuais de produtos derivados
do tabaco
### Modulo Q - Doenças Cronicas (por UF)
Tabelas:\
3.2. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
hipertensão arterial\
3.14. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
diabetes\
3.28. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
colesterol alto\
3.29. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
alguma doença do coração\
3.34. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
asma\
3.39. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
artrite ou reumatismo\
3.40. Proporção de pessoas de 18 anos ou mais de idade que referiram problema crônico de
coluna\
3.44. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico de
depressão por profissional de saúde mental\
3.52. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico médico de
câncer\
3.55. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico de
insuficiência renal crônica\
3.56. Proporção de pessoas de 18 anos ou mais de idade que referiram diagnóstico de
insuficiência renal crônica em hemodiálise/diálise

In [11]:
strip_accents = lambda s: ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

#### MODULO Q
p = PATH_EXTDB + "ibge\\pesquisa-nacional-saude_2019\\Modulo_Q.zip"
files = ['3.2','3.14','3.28','3.29','3.34','3.39','3.40','3.44','3.52','3.55','3.56']
feat_names = ['HIPERTENS','DIABETES','COLEST','CARD','ASMA','REUMA','COLUNA','DEPRE','CANCER','RENAL','RENALHEMOD']

dfs = []
with ZipFile(p) as z:
    for fl, feat in zip(files, feat_names):
        with z.open(fl+'.xls') as f:  
            data = f.read()
            dfs.append(pd.read_excel(data, sheet_name='sexo', header=None, names=['UF','PROP_{}'.format(feat)], usecols="A,B", skiprows=9))

# drop nan
dfs = list(map(lambda df: df.dropna(axis='index', how='any'), dfs))

# remove accents from UF column, set UF column as index and select only the values regarding Brazilian UF 
# (according to ID_UF in <uf general info> table)
def adjust_df(df):
    df.UF = df.UF.apply(strip_accents)
    return df.set_index('UF').loc[ID_UF].copy()
dfs = [adjust_df(df.copy()) for df in dfs]
df_q = reduce(lambda df1, df2: df1.join(df2), dfs)


#### MODULO N & P
dfs = []

# MODULO N: really good health
p = PATH_EXTDB + "ibge\\pesquisa-nacional-saude_2019\\Modulo_N.zip"
with ZipFile(p) as z:
    with z.open('1.1.xls') as f:  
        data = f.read()
        dfs.append(pd.read_excel(data, sheet_name='sexo', header=None, names=['UF','PROP_SAUDMTBOA'.format(feat)], usecols="A,B", skiprows=9))

# MODULO P: physical activity and tabagism
p = PATH_EXTDB + "ibge\\pesquisa-nacional-saude_2019\\Modulo_P.zip"
z = ZipFile(p)
with z.open('3 - Atividade Física/2.18.xls') as f:  
    data = f.read()
    dfs.append(pd.read_excel(data, sheet_name='sexo', header=None, names=['UF','PROP_EXFISICO'.format(feat)], usecols="A,B", skiprows=9))
with z.open('4 - Tabagismo/2.26.xls') as f:  
    data = f.read()
    dfs.append(pd.read_excel(data, sheet_name='sexo', header=None, names=['UF','PROP_TABACO'.format(feat)], usecols="A,B", skiprows=9))
    
# drop nan
dfs = list(map(lambda df: df.dropna(axis='index', how='any'), dfs))

# remove accents from UF column, set UF column as index and select only the values regarding Brazilian UF 
# (according to ID_UF in <uf general info> table)
def adjust_df(df):
    df.UF = df.UF.apply(lambda s: ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))
    return df.set_index('UF').loc[ID_UF].copy()
dfs = [adjust_df(df.copy()) for df in dfs]
df_np = reduce(lambda df1, df2: df1.join(df2), dfs)


#### UNIFY ALL MODULES
df_pns = df_np.join(df_q)

In [12]:
df_pns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Acre to Tocantins
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PROP_SAUDMTBOA   27 non-null     float64
 1   PROP_EXFISICO    27 non-null     float64
 2   PROP_TABACO      27 non-null     float64
 3   PROP_HIPERTENS   27 non-null     float64
 4   PROP_DIABETES    27 non-null     float64
 5   PROP_COLEST      27 non-null     float64
 6   PROP_CARD        27 non-null     float64
 7   PROP_ASMA        27 non-null     float64
 8   PROP_REUMA       27 non-null     float64
 9   PROP_COLUNA      27 non-null     float64
 10  PROP_DEPRE       27 non-null     float64
 11  PROP_CANCER      27 non-null     float64
 12  PROP_RENAL       27 non-null     float64
 13  PROP_RENALHEMOD  27 non-null     float64
dtypes: float64(14)
memory usage: 4.4+ KB


In [13]:
df_pns.head()

,PROP_SAUDMTBOA,PROP_EXFISICO,PROP_TABACO,PROP_HIPERTENS,PROP_DIABETES,PROP_COLEST,PROP_CARD,PROP_ASMA,PROP_REUMA,PROP_COLUNA,PROP_DEPRE,PROP_CANCER,PROP_RENAL,PROP_RENALHEMOD
UF,,,,,,,,,,,,,,
Acre,59.9,28.9,15.1,19.2,4.3,9.8,3.5,3.7,7.2,16.5,6.0,1.2,0.8,8.3
Alagoas,55.9,27.5,10.6,23.9,7.8,14.2,4.0,3.8,7.4,21.1,6.2,1.5,1.3,12.4
Amapa,58.9,36.1,10.9,18.2,4.3,10.4,2.0,6.5,7.6,15.0,4.5,1.2,1.0,2.0
Amazonas,68.2,30.2,10.2,16.0,5.4,10.9,2.6,4.5,6.5,19.8,4.2,1.2,0.8,5.3
Bahia,54.2,30.1,10.1,25.2,6.7,14.7,4.0,2.9,7.0,26.0,6.3,1.9,0.7,8.8


# ===================
#   ATLASBRASIL-ORG
# ===================
#### Censo 2010 (por município)

In [14]:
p = PATH_EXTDB + "atlasbrasil-org\\Atlas 2013_municipal, estadual e Brasil.xls"
rename = {'Codmun6':'CO_MUN_NODIG', 'Codmun7':'CO_MUN', 'Município':'ID_MUN'}
cols2use = [0,1,2,3,4,9,13,18,70,71,72,73,74,75,76,78,80,91,99,100,101,102,126,132,138,140,143,148,149,233,234,235,236]
df = pd.read_excel(p, sheet_name=1, usecols=cols2use)
df = df[df.ANO==2010].reset_index(drop=True)
df.rename(columns=rename, inplace=True)

In [15]:
df.head()

,ANO,UF,CO_MUN_NODIG,CO_MUN,ID_MUN,RAZDEP,E_ANOSESTUDO,T_ANALF18M,CORTE1,CORTE2,...,T_DES18M,T_AGUA,T_DENS,AGUA_ESGOTO,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,IDHM,IDHM_E,IDHM_L,IDHM_R
0,2010,11,110001,1100015,ALTA FLORESTA D'OESTE,47.37,8.67,13.00,102.4,224.80,...,5.17,93.69,22.58,9.31,40.27,24.81,0.641,0.526,0.763,0.657
1,2010,11,110002,1100023,ARIQUEMES,46.68,9.18,8.53,200.0,335.00,...,4.57,98.54,27.15,11.40,26.59,12.27,0.702,0.600,0.806,0.716
2,2010,11,110003,1100031,CABIXI,48.12,10.09,14.73,130.0,233.33,...,2.58,95.49,19.93,3.31,38.52,21.53,0.650,0.559,0.757,0.650
3,2010,11,110004,1100049,CACOAL,43.79,9.74,8.94,200.0,336.67,...,5.61,97.96,20.50,1.85,26.16,11.58,0.718,0.620,0.821,0.727
4,2010,11,110005,1100056,CEREJEIRAS,45.67,10.22,11.17,172.5,303.33,...,5.42,97.53,16.66,5.34,32.21,15.53,0.692,0.602,0.799,0.688


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ANO                   5565 non-null   int64  
 1   UF                    5565 non-null   int64  
 2   CO_MUN_NODIG          5565 non-null   int64  
 3   CO_MUN                5565 non-null   int64  
 4   ID_MUN                5565 non-null   object 
 5   RAZDEP                5565 non-null   float64
 6   E_ANOSESTUDO          5565 non-null   float64
 7   T_ANALF18M            5565 non-null   float64
 8   CORTE1                5565 non-null   float64
 9   CORTE2                5565 non-null   float64
 10  CORTE3                5565 non-null   float64
 11  CORTE4                5565 non-null   float64
 12  CORTE9                5565 non-null   float64
 13  GINI                  5565 non-null   float64
 14  PIND                  5565 non-null   float64
 15  PMPOB                